<a href="https://www.kaggle.com/code/ahksase/abdulrahman-hesham-capstone-project-task-2?scriptVersionId=144972014" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pixellib

In [ ]:
!/opt/conda/bin/python3.7 -m pip install --upgrade pip

In [ ]:
!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

In [ ]:
import pixellib
from pixellib.instance import instance_segmentation
import cv2

segment_video = instance_segmentation()
segment_video.load_model("mask_rcnn_coco.h5")
target_classes = segment_video.select_target_classes(car = True)
segment_video.process_video("/kaggle/input/tahrir-square/TahrirSquare.mp4", show_bboxes = True, frames_per_second= 20, segment_target_classes= target_classes,  output_video_name="AbdulRahman_Hesham_Capstone_Project_Task_2.mp4")

In [ ]:
import pixellib
from pixellib.instance import instance_segmentation
import cv2

model = instance_segmentation()
model.load_model("mask_rcnn_coco.h5")
target_classes = model.select_target_classes(car=True)
video_file = "/kaggle/input/tahrir-square/TahrirSquare.mp4"
cap = cv2.VideoCapture(video_file)

# Get the video's frames per second (fps) and frame dimensions
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_file = "AbdulRahman_Hesham_Capstone_Project_Task_2.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_file, fourcc, 20, (frame_width, frame_height))

# Initialize the car counter
Car_Count = 0
i = 1

# Loop over the frames in the video
while cap.isOpened():
    # Get the next frame
    ret, frame = cap.read()
    if frame is None:
        break
    
    # Segment the frame
    segmentation_result = model.segmentFrame(frame, verbose=0,segment_target_classes= target_classes,show_bboxes = True)
    class_ids = segmentation_result[0]['class_ids'].astype(int)
    
    # Count the number of cars in the frame
    for class_id in class_ids:
        if class_id == 3:
            Car_Count += 1
    
    # Put the car count on the frame
    cv2.putText(frame, f"Car Count: {Car_Count}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)    
    # Write the frame to the output video
    out.write(frame)
    
    print(f"No of frames:{i}")
    i += 1
    Car_Count = 0
    
    # Press `q` to quit
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

# Release VideoWriter and close the video file
out.release()
cap.release()